## 0. install & import

In [ ]:
!pip install google-api-python-client
!pip install google-auth google-auth-httplib2 google-auth-oauthlib
!pip install YoutubeTags
!pip install -U sentence-transformers

#자막 추출
!pip install youtube-transcript-api

# textrank 패키지 설치
!pip install git+https://github.com/lovit/textrank.git

In [ ]:
#mecab 돌릴 때 실행시키면됨
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
import numpy as np
from konlpy.tag import Kkma
import os
import re
import pandas as pd
from konlpy.tag import Okt
from konlpy.tag import Mecab
from textrank import KeywordSummarizer

from torch import Tensor
from sentence_transformers import SentenceTransformer, util

# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')

In [ ]:
# 유튜브 관련 함수들
from googleapiclient.discovery import build
from youtube_transcript_api import YouTubeTranscriptApi
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser

os.environ['GOOGLE_API_KEY'] = # 개인 유튜브 API key

DEVELOPER_KEY = # 개인 유튜브 API key
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey= DEVELOPER_KEY)

## 1. 관련 함수들 정의

In [ ]:
# 키워드를 추출하기 위해서 먼저 단어 그래프를 만들어야함
# sents 는 list of str 형식의 문장들이며, tokenize 는 str 형식의 문장을 list of str 형식의 단어열로 나누는 토크나이저

from collections import Counter

def scan_vocabulary(sents, tokenize, min_count=2):
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

In [ ]:
# textRank 에서 두 단어 간의 유사도를 정의하기 위해 두 단어의 co-occurrence 계산

from collections import defaultdict

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2):
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    return dict_to_mat(counter, n_vocabs, n_vocabs)

In [ ]:
# dict_to_mat 함수를 이용해 sparse matrix로 변환한 후 반환
from scipy.sparse import csr_matrix

def dict_to_mat(d, n_rows, n_cols):
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

In [ ]:
# word graph 함수 만들기
def word_graph(sents, tokenize=None, min_count=2, window=2, min_cooccurrence=1):
    idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    tokens = [tokenize(sent) for sent in sents]
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence, verbose)
    return g, idx_to_vocab

In [ ]:
# PageRank 를 학습하는 함수
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30):
    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)
    bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

In [ ]:
# textrank_keyword 함수

def textrank_keyword(sents, tokenize, min_count, window, min_cooccurrence, df=0.85, max_iter=30, topk=30):
    g, idx_to_vocab = word_graph(sents, tokenize, min_count, window, min_cooccurrence)
    R = pagerank(g, df, max_iter).reshape(-1)
    idxs = R.argsort()[-topk:]
    keywords = [(idx_to_vocab[idx], R[idx]) for idx in reversed(idxs)]
    return keywords

In [ ]:
# 불용어

stopwords_k = ['은','는','아','안','전','얘','휴','유','박','거','것','어','속','나','을','를','게','우','에','의','이','가',
               '이','때','오','우','으로','음악','로','에게','에서','까지','께','저','도','한','그리고', '제가','로써','로서',
               '지금','이제','으로써','다','등','등등','들','제','까지','좀','사실','조금','몇','하면','다면','와','과','왜',
               '나','그','때','어느','하다','네','뭐','네네','중인','만큼','진짜', '마찬가지', "아", "휴", "그냥","아이구",
               "아이쿠", "아이고", "우리", "저희", "따라", "의해",'이제', '남자', '여자', '말','이야기','얘기','사람','생각']

## 2. 유튜브 API 크롤링

In [ ]:
# Okt

okt = Okt()

def okt_tokenizer(sent):
    words = okt.pos(sent, join=True)
    words = [w for w in words if ('/Noun' in w)]
    return words

In [ ]:
# Mecab

mecab = Mecab()

def mecab_tokenizer(sent):
    words = mecab.pos(sent, join=True)
    words = [w for w in words if ('/NNP' in w or '/NNG' in w or '/XR' in w)]
    return words

In [ ]:
# 깔끔하게 스크립트 떼어오기

def get_script(url):

  TOKENIZER = mecab_tokenizer

  srt = YouTubeTranscriptApi.get_transcript(url, languages=["ko"])

  lines = ""
  for txt in srt:
    lines = lines + txt['text'] + " "

  return lines

In [ ]:
# keyword 추출 함수

def get_keywords(url):
  TOKENIZER = mecab_tokenizer

  srt = YouTubeTranscriptApi.get_transcript(url, languages=["ko"])
  with open("subtitles.txt", "w", encoding="utf-8") as f:
      for i in srt:
          f.write("{}\n".format(i))

  with open("subtitles.txt", "r", encoding="utf-8") as f:
      sents = f.read()

  with open('subtitles_mecab.txt', 'w') as f:
      f.writelines(TOKENIZER(sents))

  with open("subtitles_mecab.txt", "r") as f:
      sents = [sent.strip() for sent in f]

  keyword_extractor = KeywordSummarizer(
      tokenize = TOKENIZER,
      window = -1,
      verbose = False
  )

  keywords = keyword_extractor.summarize(sents, topk=20)

  cnt = 0
  idx_list = []
  for word, rank in keywords:
      if (word.split("/")[0] not in stopwords_k) :
        idx_list.append(cnt)
      cnt = cnt + 1

  keywords_value = list(pd.Series(keywords)[idx_list])

  return keywords_value

In [ ]:
# 일반 문장에서 키워드 추출 함수
# input parameter(sentence) 형식 : 전체 문장이 하나의 문자열 안에 들어간 연결된 형태

def get_keywords_for_sentences(sentence):

  # 토크나이즈 및 추출함수 정의
  TOKENIZER = mecab_tokenizer

  keyword_extractor = KeywordSummarizer(
      tokenize = TOKENIZER,
      window = -1,
      verbose = False
  )

  # 문장 토크나이즈
  sents = TOKENIZER(sentence)
  sentence_sum = ''
  for sent in sents:
    sentence_sum = sentence_sum + sent # 하나의 문자열로 병합

  # 키워드 추출
  keywords = keyword_extractor.summarize([sentence_sum], topk=20)

  # 불용어 제거 및 병합
  cnt = 0
  idx_list = []
  for word, rank in keywords:
      if (word.split("/")[0] not in stopwords_k) :
        idx_list.append(cnt)
      cnt = cnt + 1

  keywords_value = list(pd.Series(keywords)[idx_list])

  return keywords_value

In [ ]:
# 태그 추출 함수

def get_tags(url):

  # 비디오 정보 가져오기
  video_info = youtube.videos().list(id=url, part="snippet").execute()

  tags = video_info["items"][0]["snippet"]["tags"]
  return tags

In [ ]:
# 제목 추출 함수

def get_title(url):

  # 비디오 정보 가져오기
  video_info = youtube.videos().list(id=url, part="snippet").execute()

  video_title = video_info["items"][0]["snippet"]["title"]
  return video_title

## 3. 분석대상 영상 선별

In [ ]:
PC_gamelist = ['가디언 테일즈', '건바운드', '검은사막', '겟앰프드', '귀혼', '그란 투리스모 7',
                '그란 투리스모 스포트', '끄투', '갈틱폰', '내 맘대로 z9별', '네이비필드', '네이비필드2',
                '능력자x', '다크에덴','더 크루 2','더 킹 오브 파이터즈 온라인 드래곤플라이판',
                '던전앤파이터', '데스티니 가디언즈', '더플레이어', '다크소울', '라그나로크', '라테일',
                '라피스', '러브비트' , '레드 데드', '로도스도 전기', '로스트사가', '로스트아크',
                '룬스케이프', '리그 오브 레전드','롤',"lol", '리프트', '로블록스',  '마구마구', '마비노기',
                '마이트 앤 매직', '마인크래프트','마크',"minecraft", '메탈기어', '메틴', '모두의마블',
                '믹스마스터', '마피아42', '메이플스토리', '메이플', '무한의 계단', '바람의 나라',
                '부활! 얍카!', '붉은 보석', '블레이드 앤 소울', '브롤스타즈', '배배배뱀', '사이퍼즈',
                '서든어택', '서든', '소울워커', '숨바꼭질', '스키드러쉬', '스타크래프트', '스타워즈',
                '스타 트렉', '슬러거', '시아', '시티레이서', '신마법의대륙', '아이모', '아이온',
                '아스가르드', '아바벨', '야채부락리', '얼음땡', '이터널시티', '어둠의 전설',
                '에오스 더 블루', '엘소드', '클럽 엠스타', '오디션', '울프', '워 썬더', '워프레임',
                '월드 오브 워크래프트', '월드 오브 탱크', '월드 오브 워플레인', '월드 오브 워쉽', '오버워치',
                '오버워치2', '좀비고등학교', '챔피언스', '카운터 스트라이크', '카트라이더','카트',
                '러쉬플러스', '칼', '캐치마인드', '코즈믹 브레이크', '코즈믹 아레나', '콜 오브 카오스',
                '쿠키런', '크레이지 아케이드','크아', '크리티카', '크로스파이어', '클로저스', '캣점프',
                '테일즈런너','테런', '테일즈위버', '토람', '포트리스', '탕탕특공대', '파이널판타지',
                '피파 온라인','피파','fifa', '핀볼도사', '판타시 스타', '팡야', '포키포키', '포트나이트',
                '프렌즈팝', '프렌즈팝콘', '프렌즈사천성', '프렌즈샷', '프렌즈마블', '프렌즈레이싱','프렌즈타워',
                '프렌즈타운', '프리스타일', '프리스톤테일', '피피루와 419개의 근원이 되는 죄', '포켓몬 고',
                '포켓몬', '하운즈', '헬무', 'C9', 'EVE', 'Fall Guys', 'Grand Theft Auto Online',
                'haven & hearth', 'mobmania', 'osu!', 'robocraft', 'rf온라인', 's4리그', 't3 아레나',
                'volcanoids','마인크래프트','디아블로','카운터스트라이크','심즈','심즈2','심즈3','와우',
                '월드오브워크래프트','워크래프트','하프라이프','배틀필드','스타크래프트','길드워','미스트',
                '심시티','리븐','코삭','파퓰러스','포탈','롤러코스터','워해머','둠','에버퀘스트','테마파크',
                '길드워','문명','발더스 게이트','심시티','던전', '스타 워즈','타이쿤','모노폴리',
                'minecraft','suddenattack','talesrunner','pokemon','롤드컵','t1', '피파4','fifa4','fifa 온라인','피파']


movie_list = ['영화리뷰','영화','드라마','리뷰','결말포함','몰아보기', '결말']

sports_list = ['맨시티','리버풀','아스널','아스날','토트넘','아스톤 빌라','맨유','맨체스터 유나이티드','뉴캐슬',
               '웨스트햄','첼시','브랜포드','울버햄튼','크리스탈팰리스','에버튼','번리',
               '레알마드리드','바르셀로나','아틀레티코 마드리드','발렌시아','뮌헨','레버쿠젠','도르트문트',
               '나폴리','인터밀란','유벤투스','AC 밀란','파리생제르맹','생제르맹','니스','PSG',
               '손흥민','이강인','김민재','조규성','황희찬','황의조','조현우','벤투','정우영','이승우','음바페','호날두','메시',
               '홀란드','네이마르','해리케인','김연경','김연아', "LG트윈스",'KTwiz',"NC다이노스",'두산베어즈',
               '기아타이거즈','롯세자이언츠','삼성라이온즈','한화이글스','키움히어로즈','박찬호','이승엽',
               '류현진','김하성','황재균','오타니','박병호','추신수','KLPGA','한국시리즈','안세영','안산', 'KBO']

OUT_list = ['vlog','브이로그', 'asmr', '먹방', 'playlist','플레이리스트', 'grwm', '겟레디윗미','겟레디위드미',
            '다꾸','커버','cover','댄스','하울'] # + 자막 없는 영상

In [ ]:
# EXAMPLE : 유튜브 영상의 링크에서 따온 고유번호
url_list = ['lF5RJqDj1xE','gRYO-L3_yO8',"HlZWDmUv_x8",'51L3tNHhDHo','lqCzth7iBHk','T_qeUTzW7-o','CaP0C4nAw_Q','fCuiNtwH7pQ','9r2_gssMPBg','j5Jc3GmX1gc']

In [ ]:
# 1. 함수 돌려서 분석 대상인 영상만 선별
# OUT_list에 속하는 단어가 제목, 태그에 있는 경우 out
# 스크립트 자체가  없는 경우 out

In [ ]:
# 스크립트 있는 url만 추출

def NO_SCRIPT(url_list):
  url_in_script_list = []
  for url in url_list:
    try:
      get_keywords(url)
      url_in_script_list.append(url)
    except:
      pass # 스크립트 없는 경우 out
  return url_in_script_list

In [ ]:
url_list = NO_SCRIPT(url_list) # 스크립트 있는 url

In [ ]:
# 문자열 그냥 넣고 해당 리스트에 있는 문자들에서 only 문자랑 숫자만 남기기

def only_word(L):
  LL = []
  for i in L:
    result = re.sub('[^0-9a-zA-Zㄱ-힗]', '', i)
    LL.append(result)
  return LL

In [ ]:
# 하나의 비디오에 대한 제목, 태그를 input으로 받아서 TF로 사용 유무 판단

def whether_each_video_in(titles, tags):
  out = 1
  for tag in tags:
    tag = tag.lower()
    if tag in OUT_list:
      out = 0
    else:
      pass

  for title in titles:
    title = title.lower()
    if title in OUT_list:
      out = 0
    else:
      pass

  return bool(out)

In [ ]:
# 위의 whether_each_video_in을 받아서 url_list 적용

def select_url(url_list):
  final_url_list = []
  for i in range(len(url_list)):
    titles = only_word(get_title(url_list[i]).split(' '))
    tags = get_tags(url_list[i])
    if whether_each_video_in(titles,tags):
      final_url_list.append(url_list[i])
  return final_url_list

In [ ]:
final_url_list = select_url(url_list) # 최종 분석에 사용할 url 리스트 선별

## 4. 정보성 영상과 오락성 영상 구분

In [ ]:
# 2. 함수 돌려서 여기에 위의 리스트 안에 있는 단어가 있는 경우 제목 제외하고 키워드만 사용

In [ ]:
# PC_gamelist, movie_list, sports_list로 판단될 단어가 제목 혹은 태그에 있는지 판단

def tag_in_TF(url):
  info_url = []
  noninfo_url = []

  tags = get_tags(url)
  for tag in tags:
    tag = tag.lower() # 영어는 전부 소문자로 변환
    if tag in PC_gamelist:
      noninfo_url.append(tag)
    elif tag in movie_list:
      noninfo_url.append(tag)
    elif tag in sports_list:
      noninfo_url.append(tag)
    else:
      info_url.append(tag)

  return info_url, noninfo_url

In [ ]:
def info_noninfo_list(final_url_list):
  info_url_list = []
  noninfo_url_list = []

  for url in final_url_list:
    tags = tag_in_TF(url)
    if len(tags[1]) == 0:
      info_url_list.append(url)
    else:
      noninfo_url_list.append(url)
  return info_url_list, noninfo_url_list

In [ ]:
# 3. info_url_list에 있는 것들 : (제목 & 키워드 유사도 ) X 키워드 점수

In [ ]:
info_url_list = info_noninfo_list(final_url_list)[0] # 정보성영상 리스트
noninfo_url_list = info_noninfo_list(final_url_list)[1] # 오락성영상 리스트

## 5. 정보성 영상 유사도 계산 후 점수화

In [ ]:
# keyword랑 title 넣으면 딱 값들만 떼어서 합친 리스트 만들어주는 함수

def make_sentence(keywords,title):
  key_array = []

  for key in keywords:
    key_array.append(key[0].split('/')[0])

  key_array.append(title)
  return key_array

In [ ]:
# 개별 url에 대한 유사도 계산

def similarity(url):
  keywords = get_keywords(url)
  title = get_title(url)

  # 여기서 sentence에 키워드 리스트로 넣고
  sentences = make_sentence(keywords,title)

  vectors = model.encode(sentences) # encode sentences into vectors
  similarities = util.cos_sim(vectors, vectors) # compute similarity between sentence vectors
  # similarities = np.abs(similarities[-1])[:len(keywords)]
  similarities = similarities[-1][:len(keywords)]

  return similarities.tolist()

In [ ]:
# keyword 점수와 유사도 바탕으로 최종 점수 계산

def get_score_info(url):
  keywords = get_keywords(url)
  keyword_score = []
  for key in keywords:
    keyword_score.append(key[1])

  # minmax 적용
  # keyword_score = keyword_score - np.min(keyword_score)
  # keyword_score = keyword_score/np.max(keyword_score)

  # keyword_score = np.log1p(keyword_score)

  simil = similarity(url)

  score_for_keys = [x * y for x, y in zip(keyword_score, np.square(simil))]

  return score_for_keys

In [ ]:
# 최종 점수 기준 상위 8개 단어 추출

def extract_keyword_FINAL(url_list):
  FINAL_keywords = []
  for url in info_url_list:
    keyword_list = make_sentence(get_keywords(url),get_title(url))[:-1]
    score_list = get_score_info(url)
    df = pd.DataFrame(score_list,keyword_list).reset_index()
    df.columns = ['key','score']
    df = df.sort_values(by=['score'], ascending=False).reset_index(drop=True)
    FINAL_keywords.append(list(df['key'][:8]))
  return FINAL_keywords

In [ ]:
# 정보성 영상에 대한 최종 키워드
extract_keyword_FINAL(info_url_list)

[['금강산', '하늘', '영상', '일제', '조선', '독립', '여성', '시대'],
 ['폭탄', '아인슈타인', '미국', '과학자', '원자', '전쟁', '이야기', '프로젝트'],
 ['우주', '물리학', '물리학자', '물리', '지구', '과학', '태양', '전자기력'],
 ['칸트', '명제', '사물', '이성', '인간', '공간', '비판', '인식']]

## 6. 오락성 영상 점수 계산

In [ ]:
# 오락성 영상 키워드에서 NNP(고유명사) 제외
def NNP_out(keywords):
  key_list = []

  for key in keywords:
    if key[0].split("/")[1] != "NNP":
      key_list.append(key)
  return key_list

In [ ]:
# 어바웃타임 영상에 대한 키워드
get_keywords("V_5qzzcGtaA&t=11s")

[('팀/NNG', 5.566480661920329),
 ('시간/NNG', 3.760823552049253),
 ('아버지/NNG', 2.406580719645945),
 ('동생/NNG', 2.180873580912064),
 ('샬롯/NNP', 1.7294593034442918),
 ('행복/NNG', 1.7294593034442918),
 ('친구/NNG', 1.5037521647104075),
 ('마음/NNG', 1.5037521647104075),
 ('여행/NNG', 1.2780450259765228),
 ('인사/NNG', 1.2780450259765228),
 ('결국/NNG', 1.2780450259765228),
 ('과거/NNG', 1.2780450259765228),
 ('날/NNG', 1.2780450259765228),
 ('힘/NNG', 1.2780450259765228),
 ('대사/NNG', 1.0523378872426383),
 ('삶/NNG', 1.0523378872426383),
 ('아이/NNG', 1.0523378872426383),
 ('연극/NNG', 1.0523378872426383)]

In [ ]:
# 인셉션 영상에 대한 키워드
NNP_out(get_keywords('psq2hvtSyMA&t=18s'))

[('꿈/NNG', 5.853795412031121),
 ('단계/NNG', 2.5264349709559992),
 ('자신/NNG', 2.288766368022053),
 ('의식/NNG', 2.288766368022053),
 ('아리아드네/NNG', 2.288766368022053),
 ('사이트/NNG', 2.051097765088116),
 ('임무/NNG', 2.051097765088116),
 ('마음/NNG', 1.5757605592202446),
 ('시간/NNG', 1.5757605592202446),
 ('팀원/NNG', 1.5757605592202446),
 ('사이토/NNG', 1.3380919562863092)]